# Finetune PE dataset for ATC

## Libraries

In [1]:
# %cd ..
# %rm -rf LLaMA-Factory
# !git clone https://github.com/hiyouga/LLaMA-Factory.git
# %cd LLaMA-Factory
# %ls
# !pip install -e .[torch,bitsandbytes]

In [2]:
# !pip uninstall -y pydantic
# !pip install pydantic==1.10.9 # 

# !pip uninstall -y gradio
# !pip install gradio==3.48.0

# !pip uninstall -y bitsandbytes
# !pip install --upgrade bitsandbytes

# !pip install tqdm
# !pip install ipywidgets
# !pip install scikit-learn

# Restart kernel afterwards.

In [26]:
import os
import ast
import json
import torch
import pickle
import pandas as pd
from tqdm.notebook import tqdm
from llamafactory.chat import ChatModel
from llamafactory.extras.misc import torch_gc
from sklearn.metrics import classification_report

In [27]:
try:    
    assert torch.cuda.is_available() is True
    
except AssertionError:
    
    print("Please set up a GPU before using LLaMA Factory...")

## Load Dataset

In [28]:
os.getcwd()

'/Utilisateurs/umushtaq/LLaMA-Factory'

In [29]:
# *** MODEL NAME ***

# base_model = "unsloth/llama-3-8b-Instruct-bnb-4bit"

with open("tmp.pkl", "rb") as fh:
        
        l = pickle.load(fh)
        base_model = l[0]
        train_dataset_name = l[1]
        test_dataset_name = l[2]

In [30]:
print(base_model, train_dataset_name, test_dataset_name)

In [31]:
dataset_dir = os.path.join(os.getcwd(), "datasets")

# *** TRAIN DATASET NAME *** #

# train_dataset_name = "PE_ATC_paragraph_train.json"
train_dataset_file = os.path.join(dataset_dir, train_dataset_name)

# *** TEST DATASET NAME *** #

# test_dataset_name = "PE_ATC_paragraph_test.json"
test_dataset_file = os.path.join(dataset_dir, test_dataset_name)

In [32]:
train_dataset_file

'/Utilisateurs/umushtaq/LLaMA-Factory/datasets/PE_ATC_paragraph_train.json'

## Fine-tune Model

In [33]:
# *** MODEL DIR ***
model_name = f"""{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}"""

train_file = os.path.join(os.getcwd(), f"""cli_files/{train_dataset_name.split(".")[0].split("train")[0]}{base_model.split("/")[1]}.json""")
output_dir = os.path.join(os.getcwd(), "models", model_name)

nb_epochs = 10 # 3

In [34]:
output_dir

'/Utilisateurs/umushtaq/LLaMA-Factory/models/PE_ATC_paragraph_llama-3-8b-Instruct-bnb-4bit'

In [35]:
model_name

'PE_ATC_paragraph_llama-3-8b-Instruct-bnb-4bit'

In [36]:
train_file

'/Utilisateurs/umushtaq/LLaMA-Factory/cli_files/PE_ATC_paragraph_llama-3-8b-Instruct-bnb-4bit.json'

In [37]:
os.path.isfile(train_file)

False

In [38]:
dataset_info_line =  {
  "file_name": f"{train_dataset_file}",
  "columns": {
    "prompt": "instruction",
    "query": "input",
    "response": "output"
  }
}

In [39]:
dataset_info_line

{'file_name': '/Utilisateurs/umushtaq/LLaMA-Factory/datasets/PE_ATC_paragraph_train.json',
 'columns': {'prompt': 'instruction', 'query': 'input', 'response': 'output'}}

In [40]:
os.getcwd()

'/Utilisateurs/umushtaq/LLaMA-Factory'

In [41]:
# with open("LLaMA-Factory/data/dataset_info.json", 'r+') as fh:
#     file_data = json.load(fh)
#     # file_data["persuasive_essays"] = dataset_info_line
#     file_data["persuasive_essays_2"] = dataset_info_line # use this if running different tasks on same node
#     fh.seek(0)
#     json.dump(file_data, fh, indent = 4)

In [42]:
with open("LLaMA-Factory/data/dataset_info.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["persuasive_essays"] = dataset_info_line

with open("LLaMA-Factory/data/dataset_info.json", "w") as jsonFile:
    json.dump(data, jsonFile)

FileNotFoundError: [Errno 2] No such file or directory: 'LLaMA-Factory/data/dataset_info.json'

In [ ]:
args = dict(
  stage="sft",                           # do supervised fine-tuning
  do_train=True,
  model_name_or_path=base_model,         # use bnb-4bit-quantized Llama-3-8B-Instruct model
  # dataset="persuasive_essays",           # use alpaca and identity datasets
  dataset="persuasive_essays_2",           # use alpaca and identity datasets
  template="llama3",                     # use llama3 prompt template
  finetuning_type="lora",                # use LoRA adapters to save memory
  lora_target="all",                     # attach LoRA adapters to all linear layers
  output_dir=output_dir,                 # the path to save LoRA adapters
  overwrite_output_dir=True,             # overrides existing output contents
  per_device_train_batch_size=2,         # the batch size
  gradient_accumulation_steps=4,         # the gradient accumulation steps
  lr_scheduler_type="cosine",            # use cosine learning rate scheduler
  logging_steps=10,                      # log every 10 steps
  warmup_ratio=0.1,                      # use warmup scheduler
  save_steps=3000,                       # save checkpoint every 1000 steps
  learning_rate=5e-5,                    # the learning rate
  num_train_epochs=nb_epochs,            # the epochs of training
  max_samples=2000,                       # use 500 examples in each dataset
  max_grad_norm=1.0,                     # clip gradient norm to 1.0
  quantization_bit=4,                    # use 4-bit QLoRA
  loraplus_lr_ratio=16.0,                # use LoRA+ algorithm with lambda=16.0
  fp16=True,                             # use float16 mixed precision training
  report_to="none"                       # discards wandb
)

In [ ]:
json.dump(args, open(train_file, "w", encoding="utf-8"), indent=2)

In [ ]:
os.getcwd()

In [ ]:
%cd LLaMA-Factory/
!set train_file = train_file

In [43]:
train_file

'/Utilisateurs/umushtaq/LLaMA-Factory/cli_files/PE_ATC_paragraph_llama-3-8b-Instruct-bnb-4bit.json'

In [44]:
!llamafactory-cli train $train_file

Traceback (most recent call last):
  File "/Utilisateurs/umushtaq/.conda/envs/llama-factory-notebook/bin/llamafactory-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/Utilisateurs/umushtaq/LLaMA-Factory/src/llamafactory/cli.py", line 93, in main
    run_exp()
  File "/Utilisateurs/umushtaq/LLaMA-Factory/src/llamafactory/train/tuner.py", line 27, in run_exp
    model_args, data_args, training_args, finetuning_args, generating_args = get_train_args(args)
                                                                             ^^^^^^^^^^^^^^^^^^^^
  File "/Utilisateurs/umushtaq/LLaMA-Factory/src/llamafactory/hparams/parser.py", line 130, in get_train_args
    model_args, data_args, training_args, finetuning_args, generating_args = _parse_train_args(args)
                                                                             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/Utilisateurs/umushtaq/LLaMA-Factory/src/llamafactory/hparams/parser.py", line 116, in _parse_trai

## Inference on the fine-tuned model

In [21]:
output_dir

'/Utilisateurs/umushtaq/models/PE_ATC_llama-3-8b-Instruct-bnb-4bit'

In [22]:
os.listdir(output_dir)

['trainer_log.jsonl',
 'README.md',
 'adapter_model.safetensors',
 'adapter_config.json',
 'tokenizer_config.json',
 'special_tokens_map.json',
 'tokenizer.json',
 'training_args.bin',
 'train_results.json',
 'all_results.json',
 'trainer_state.json']

In [23]:
# %cd /content/LLaMA-Factory/

args = dict(
  model_name_or_path=base_model, # use bnb-4bit-quantized Llama-3-8B-Instruct model
  adapter_name_or_path=output_dir,            # load the saved LoRA adapters
  template="llama3",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  quantization_bit=4,                    # load 4-bit quantized model
)


In [24]:
model = ChatModel(args)

[INFO|tokenization_utils_base.py:2108] 2024-06-08 12:48:19,398 >> loading file tokenizer.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/2950abc9d0b34ddd43fd52bbf0d7dca82807ce96/tokenizer.json
[INFO|tokenization_utils_base.py:2108] 2024-06-08 12:48:19,400 >> loading file added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:2108] 2024-06-08 12:48:19,402 >> loading file special_tokens_map.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/2950abc9d0b34ddd43fd52bbf0d7dca82807ce96/special_tokens_map.json
[INFO|tokenization_utils_base.py:2108] 2024-06-08 12:48:19,404 >> loading file tokenizer_config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/2950abc9d0b34ddd43fd52bbf0d7dca82807ce96/tokenizer_config.json
[WARNING|logging.py:314] 2024-06-08 12:48:19,861 >> Sp

06/08/2024 12:48:19 - INFO - llamafactory.data.template - Replace eos token: <|eot_id|>


/Utilisateurs/umushtaq/.conda/envs/llama-factory-notebook/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO|configuration_utils.py:733] 2024-06-08 12:48:19,990 >> loading configuration file config.json from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/2950abc9d0b34ddd43fd52bbf0d7dca82807ce96/config.json
[INFO|configuration_utils.py:796] 2024-06-08 12:48:19,993 >> Model config LlamaConfig {
  "_name_or_path": "unsloth/llama-3-8b-Instruct-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size"

06/08/2024 12:48:19 - INFO - llamafactory.model.utils.quantization - Loading ?-bit BITSANDBYTES-quantized model.
06/08/2024 12:48:19 - INFO - llamafactory.model.patcher - Using KV cache for faster generation.


[WARNING|quantization_config.py:393] 2024-06-08 12:48:20,644 >> Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
[INFO|modeling_utils.py:3474] 2024-06-08 12:48:20,653 >> loading weights file model.safetensors from cache at /Utilisateurs/umushtaq/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct-bnb-4bit/snapshots/2950abc9d0b34ddd43fd52bbf0d7dca82807ce96/model.safetensors
[INFO|modeling_utils.py:1519] 2024-06-08 12:48:21,503 >> Instantiating LlamaForCausalLM model under default dtype torch.bfloat16.
[INFO|configuration_utils.py:962] 2024-06-08 12:48:21,512 >> Generate config GenerationConfig {
  "bos_token_id": 128000,
  "eos_token_id": 128009
}

[INFO|modeling_utils.py:4280] 2024-06-08 12:48:52,674 >> All model checkpoint weights were used when initializing LlamaForCausalLM.

[INFO|modeling_utils.py:4288] 2024-06-08 12:48:52,677 >> All the weights of LlamaForCau

06/08/2024 12:48:53 - INFO - llamafactory.model.utils.attention - Using torch SDPA for faster training and inference.
06/08/2024 12:48:53 - INFO - llamafactory.model.adapter - Upcasting trainable params to float32.
06/08/2024 12:48:53 - INFO - llamafactory.model.adapter - Fine-tuning method: LoRA
06/08/2024 12:48:54 - INFO - llamafactory.model.adapter - Loaded adapter(s): /Utilisateurs/umushtaq/models/PE_ATC_llama-3-8b-Instruct-bnb-4bit
06/08/2024 12:48:54 - INFO - llamafactory.model.loader - all params: 8051232768


In [25]:
with open(test_dataset_file, "r+") as fh:
    test_dataset = json.load(fh)

In [26]:
test_prompts = []
test_grounds = []

for sample in test_dataset:
    test_prompts.append("\nUser:" + sample["instruction"] + sample["input"])
    test_grounds.append(sample["output"])

In [27]:
test_predictions = []

for prompt in tqdm(test_prompts):

    messages = []
    messages.append({"role": "user", "content": prompt})

    response = ""
    
    for new_text in model.stream_chat(messages):
        #print(new_text, end="", flush=True)
        response += new_text
        #print()
    test_predictions.append({"role": "assistant", "content": response})

    torch_gc()

In [28]:
# next(model.engine.model.parameters()).is_cuda

In [29]:
# len(test_predictions)

In [30]:
# test_predictions

In [32]:
with open(os.path.join(output_dir, f"""PE_ATC_results_{nb_epochs}.pickle"""), 'wb') as fh:
    results_d = {"ground_truths": test_grounds,
                 "predictions": test_predictions    
        
    }
    pickle.dump(results_d, fh)

## Post-processing

In [76]:
# import pickle
# import os
# import pandas as pd
# import json
# import ast
# from sklearn.metrics import classification_report

In [2]:
#'/Utilisateurs/umushtaq/models/PE_ATC_llama-3-8b-Instruct-bnb-4bit'
# output_dir = 'models/PE_ATC_essay_llama-3-8b-Instruct-bnb-4bit'
# nb_epochs = 10

In [3]:
with open(os.path.join(output_dir, f"""PE_ATC_results_{nb_epochs}.pickle"""), "rb") as fh:
        
        results = pickle.load(fh)

In [4]:
grounds = results["ground_truths"]

In [5]:
preds = results["predictions"]

In [6]:
grounds = [json.loads(x)["component_types"] for x in grounds]

In [7]:
preds = [x["content"] for x in preds]

In [ ]:
preds = [json.loads(x)["component_types"] for x in preds]

In [23]:
def opposite(component_type):

    if component_type == "Premise":
        return "Claim"
    elif component_type == "Claim":
        return "Premise"
    elif component_type == "MajorClaim":
        return "Claim"

In [24]:
def harmonize_preds(grounds, preds):

    l1, l2 = len(preds), len(grounds)
    if l1 < l2:
        diff = l2 - l1
        preds = preds + [opposite(x) for x in grounds[l1:]]
    else:
        preds = preds[:l2]
        
    return preds 

In [25]:
for i,(x,y) in enumerate(zip(grounds, preds)):
    
    if len(x) != len(y):
        
        preds[i] = harmonize_preds(x, y)

In [26]:
ATC_preds = [item for row in preds for item in row]
ATC_grounds = [item for row in grounds for item in row]

In [27]:
# sanity check: 
len(ATC_preds) == len(ATC_grounds)

True

## Results

In [28]:
print(classification_report(ATC_grounds, ATC_preds, digits=3))

              precision    recall  f1-score   support

       Claim      0.788     0.788     0.788       283
  MajorClaim      0.973     0.935     0.954       154
     Premise      0.923     0.931     0.927       724

    accuracy                          0.897      1161
   macro avg      0.895     0.885     0.890      1161
weighted avg      0.897     0.897     0.897      1161



In [29]:
with open(f"""{output_dir}/classification_report.pickle""", 'wb') as fh:
    
    pickle.dump(classification_report(ATC_grounds, ATC_preds, output_dict=True), fh)

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()